# Building LLM applications: Notebook 08

# RAG

## Initialize

In [1]:
import os
import dotenv

from pathlib import Path

from langchain_ollama import ChatOllama, OllamaEmbeddings
from langchain_chroma import Chroma
from langchain_ollama import ChatOllama
from langchain_core.prompts import PromptTemplate

In [2]:
MODEL = 'llama3.2:3b-instruct-fp16'
# MODEL = 'llama3.3:70b-instruct-fp16'
EMBEDING_MODEL = 'nomic-embed-text'

DATA_DIR = Path('.') / 'data'
CHROMA_PATH = DATA_DIR / 'chroma'
DOCS_DIR = DATA_DIR / 'books'

TOP_K = 5
SCORE_THRESHOLD = 0.7

_db_ids = None

In [3]:
# Read fro `.env` file
dotenv.load_dotenv()

OLLAMA_URL = os.getenv('OLLAMA_URL')
print(f"Using Ollama server: {OLLAMA_URL if OLLAMA_URL else 'local'}")

Using Ollama server: http://kqrw311-g5-12xlarge-a.img.astrazeneca.net:8080


## RAG

In [4]:
embedding = OllamaEmbeddings(model=EMBEDING_MODEL, base_url=OLLAMA_URL)
vdb = Chroma(persist_directory=str(CHROMA_PATH), embedding_function=embedding)

### Step 1: Query the vector database

In [5]:
question = "Who is Zaphod Beeblebrox?"


results = vdb.similarity_search_with_relevance_scores(question, k=TOP_K)

for doc, score in results:
    print(f"Score: {score}")
    print(f"Document: {doc.id}")
    print(f"Text: {doc.page_content}")
    print()

Score: 0.7474655246167894
Document: hhgttg-328
Text: Zaphod Beeblebrox,  adventurer,  ex-hippy,  good  timer,  (crook?
quite  possibly),  manic self-publicist, terribly bad at personal
relationships, often thought to be completely out to lunch.

Score: 0.5882808817417512
Document: hhgttg-991
Text: "What do you mean you've  met?"  he  demanded.  "This  is  Zaphod
Beeblebrox from Betelgeuse Five you know, not bloody Martin Smith
from Croydon."

Score: 0.5866329399649752
Document: hhgttg-878
Text: "... and news brought to you here  on  the  sub-etha  wave  band,
broadcasting  around  the  galaxy  around  the clock," squawked a
voice, "and we'll be saying a big hello to all  intelligent  life
forms  everywhere  ... and to everyone else out there, the secret
is to bang the rocks together, guys. And of course, the big  news
story  tonight  is the sensational theft of the new Improbability
Drive prototype ship by none other than Galactic President Zaphod
Beeblebrox. And the question everyone'

### Step 2: Add the results to the LLM prompt

In [6]:
prompt_template = """
Given the following information, answer the quesiton below:

{documents}

Answer the quesiton below based on the previous information:

{question}

"""

# Create a prompt object from the string template
prompt = PromptTemplate.from_template(prompt_template)


# Create a "chain" of runnable objects
llm = ChatOllama(model=MODEL, base_url=OLLAMA_URL)
runnable = prompt | llm

### Run it

In [7]:
# Join the results into a single string
docs = "\n---\n".join([doc.page_content for doc, score in results if score < SCORE_THRESHOLD])

# Invoke the llm model with the question and the documents
ret = runnable.invoke({'question': question, 'documents': docs})

ret.pretty_print()

================================== Ai Message ==================================

Based on the provided text, Zaphod Beeblebrox is the Galactic President of Betelgeuse Five who has stolen the new Improbability Drive prototype ship. He is also known for inventing the Pan Galactic Gargle Blaster, and has been described as an ex-confidence trickster with a long list of awards, including being voted the Worst Dressed Sentient Being in the Known Universe seven times.
